In [1]:
# リスト 2.2.3 MeCab 拡張辞書利用前
# ------------------------------------------------------------------------------------------------
# 目的:
#   ・拡張辞書（例: mecab-ipadic-NEologd）を使う前の“ベースライン”として、標準の辞書設定で
#     形態素解析したときの出力を確認する。
#   ・後で拡張辞書を適用した結果と比較し、固有表現（新語・地名・列車名など）や複合語の
#     分割差・品詞差を評価するための基準をつくる。
#
# 背景・理論メモ（挙動は変えない）:
#   ・MeCab は Viterbi アルゴリズムにより、辞書に登録された形態素と連接コスト表を用いて
#     最小コストの分かち書き＋品詞系列を決定する。したがって出力は「使用辞書」に強く依存する。
#   ・mecab-python3 の Tagger() は、明示指定がなければ“環境の既定辞書”を用いる。
#     既定辞書は OS/インストール手順に依存（IPADIC/UniDic/UniDic-lite など）。再現性のためには
#     「どの辞書・どの版を使ったか」を記録し、必要に応じて `Tagger("-d /path/to/dic")` で固定する。
#   ・本例の文「令和元年6月1日に特急はくたかに乗ります。」には、
#       - 年号表現（“令和元年6月1日”） … 日付のまとまりとして扱えるか
#       - 列車名（“はくたか”）         … 固有名詞として 1 トークンにまとまるか
#     といった“辞書カバレッジ”の影響が出やすい要素が含まれる。拡張辞書（NEologd 等）は
#     新語・固有表現の網羅性が高く、標準辞書より一語として認識されやすい傾向がある。
#   ・`Tagger().parse(text)` の戻り値はテキスト（行ごと: 「表層\tfeature…」＋末尾の `EOS` 行）。
#     学習用に扱う場合は、BOS/EOS 行の除去、feature のパース（CSV分解）、原形・品詞の抽出などを
#     別レイヤで行うのが一般的。本スニペットでは“生の可視化”を優先し、そのまま出力する。
#   ・比較実験の作法:
#       1) 本コード（拡張前）で出力を保存
#       2) `Tagger("-d /path/to/mecab-ipadic-neologd")` 等で拡張辞書を指定して同文を解析
#       3) 分割境界・品詞・原形の差分を検出（例: トークン列の LCS、Jaccard、編集距離 等）
#     これにより拡張辞書の効果（OOV 低減・固有表現の一語化）が定量化できる。
# ------------------------------------------------------------------------------------------------

# 辞書拡張前の結果（＝環境既定の辞書での解析）
import MeCab

# 解析対象文（年号＋日付＋列車名を含むため、辞書カバレッジの差が出やすい例）
text = "令和元年6月1日に特急はくたかに乗ります。"

# 解析実行
# - 明示的な辞書指定（-d）は行わない＝“拡張辞書利用前”のベースライン。
# - 返り値は1行1形態素のテキスト（表層\tfeature...）＋最後に "EOS" 行。
tagger1 = MeCab.Tagger()
print(tagger1.parse(text))

令和	レーワ	レイワ	令和	名詞-固有名詞-一般			1,0
元年	ガンネン	ガンネン	元年	名詞-普通名詞-副詞可能			1
6	6	6	6	名詞-数詞			0
月	ガツ	ガツ	月	名詞-普通名詞-助数詞可能			1
1	1	1	1	名詞-数詞			0
日	カ	カ	日	接尾辞-名詞的-助数詞			
に	ニ	ニ	に	助詞-格助詞			
特急	トッキュー	トッキュウ	特急	名詞-普通名詞-一般			0
は	ワ	ハ	は	助詞-係助詞			
く	ク	クル	来る	動詞-非自立可能	カ行変格	連体形-省略	1
た	タ	タ	た	助動詞	助動詞-タ	終止形-一般	
か	カ	カ	か	助詞-副助詞			
に	ニ	ニ	に	助詞-格助詞			
乗り	ノリ	ノル	乗る	動詞-一般	五段-ラ行	連用形-一般	0
ます	マス	マス	ます	助動詞	助動詞-マス	終止形-一般	
。			。	補助記号-句点			
EOS



In [3]:
# リスト2.2.4 MeCab拡張辞書利用後
# ------------------------------------------------------------------------------------------------
# 目的:
#   ・拡張辞書（例: mecab-ipadic-NEologd）を明示指定して解析し、拡張前（既定辞書）との
#     分割境界・品詞付与の違いを観察する（固有表現・新語・複合語の一語化に注目）。
#
# 背景・理論メモ（挙動は変えずコメントのみ追加）:
#   ・MeCab の出力は「使用辞書」と「連接コスト（コスト表）」に強く依存する。
#     NEologd は Web 新語や固有名詞を大量収録しており、既定辞書より OOV（未知語）を減らし、
#     例えば「はくたか」「令和元年」等のまとまりを 1 トークン化しやすい。
#   ・Tagger("-d <path>") で辞書ディレクトリを固定すると、環境差（OS/検索パス）を超えて
#     出力の再現性（determinism）が上がる。研究用途では「辞書パス」「辞書バージョン」を
#     必ず記録する（ログやREADMEに明記）。
#   ・辞書パスは環境により異なる。代表例:
#       - macOS(brew): /usr/local/lib/mecab/dic/mecab-ipadic-neologd  または  /opt/homebrew/lib/...
#       - Linux(手動インストール): /usr/lib/mecab/dic/mecab-ipadic-neologd  等
#     不明な場合はシェルで `mecab-config -D` や `find / -name mecab-ipadic-neologd -type d` で探索。
#   ・`parse()` の戻りはテキスト（1形態素1行:「表層\tfeature,...」＋末尾 EOS）。学習で使う場合は
#     BOS/EOS 除去、feature のCSV分解、原形/品詞の抽出を別レイヤで行う。
#   ・比較実験の作法（推奨）:
#       1) 既定辞書（前リスト）で解析し、結果を保存
#       2) 本リスト（NEologd 指定）で同文を解析
#       3) トークン列の差分（編集距離/Jaccard/LCS）や OOV 率の比較で定量評価
#   ・エラー対処:
#       - 「dictionary not found」→ パス不正/未インストール。辞書実体の存在を確認。
#       - パスに空白が含まれる場合は、-d 引数を引用部ごと 1 文字列にする（ここでは既に1文字列）。
# ------------------------------------------------------------------------------------------------

# 辞書拡張後の結果（NEologd を明示指定して解析）
import MeCab

# 解析対象文（年号＋日付＋列車名を含み、拡張辞書の効果が出やすい例）
text = "令和元年6月1日に特急はくたかに乗ります。"

# 解析実行
# - "-d <dir>" で辞書ディレクトリを明示指定。環境に合わせて適切なパスに置き換えること。
# - 戻り値はテキスト（形態素ごとに1行）＋末尾 "EOS" 行。可視確認を優先しそのまま print。
tagger2 = MeCab.Tagger("-d /opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd")
# chasen = MeCab.Tagger()  # 参考: 既定辞書での比較用に残している（未使用）。比較時は両者の出力を保存して差分を取る。
print(tagger2.parse(text))

令和元年	名詞,固有名詞,一般,*,*,*,2019年,レイワガンネン,レイワガンネン
6月1日	名詞,固有名詞,一般,*,*,*,6月1日,ロクガツツイタチ,ロクガツツイタチ
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
特急	名詞,一般,*,*,*,*,特急,トッキュウ,トッキュー
はくたか	名詞,固有名詞,一般,*,*,*,はくたか,ハクタカ,ハクタカ
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
乗り	動詞,自立,*,*,五段・ラ行,連用形,乗る,ノリ,ノリ
ます	助動詞,*,*,*,特殊・マス,基本形,ます,マス,マス
。	記号,句点,*,*,*,*,。,。,。
EOS

